In [ ]:
import torch
import cv2
import numpy as np
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize MTCNN (Face Detection)
mtcnn = MTCNN(thresholds=[0.7, 0.8, 0.8], device=device)  # Higher thresholds reduce false positives

# Initialize Inception Resnet (Face Recognition)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Load the webcam
cap = cv2.VideoCapture(0)

# Image preprocessing transform
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Directory for storing embeddings
embeddings_dir = 'face_embeddings'
os.makedirs(embeddings_dir, exist_ok=True)

# Load existing face embeddings
def load_embeddings():
    embeddings = {}
    for filename in os.listdir(embeddings_dir):
        if filename.endswith('.pth'):
            name = os.path.splitext(filename)[0]
            path = os.path.join(embeddings_dir, filename)
            embeddings[name] = torch.load(path, map_location=device)
    return embeddings

# Function to check if face already exists
def face_exists(new_embedding, saved_embeddings, threshold=0.7):
    if not saved_embeddings:
        return False, None
    
    similarities = {}
    for name, embedding in saved_embeddings.items():
        similarity = torch.nn.functional.cosine_similarity(new_embedding, embedding).mean()
        similarities[name] = similarity.item()
    
    best_match = max(similarities, key=similarities.get, default=None)
    if best_match and similarities[best_match] > threshold:
        return True, best_match
    return False, None

# Function to save face embedding
def save_face_embedding(face_tensor, name, saved_embeddings):
    with torch.no_grad():
        embedding = model(face_tensor.to(device)).detach()
    
    exists, existing_name = face_exists(embedding, saved_embeddings)
    if exists:
        print(f"Face already exists as '{existing_name}'")
        return False, existing_name
    
    name = ''.join(c for c in name if c.isalnum())  # Sanitize name
    save_path = os.path.join(embeddings_dir, f"{name}.pth")
    torch.save(embedding, save_path)
    print(f"Face embedding for '{name}' saved successfully!")
    return True, None

# Load existing embeddings
saved_embeddings = load_embeddings()
print(f"Loaded {len(saved_embeddings)} existing face embeddings")

# State variables
save_mode = False
input_name = ""
current_face_tensor = None
show_duplicate_alert = False
duplicate_name = ""
alert_start_time = 0
alert_duration = 3  # seconds

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    display_frame = frame.copy()
    current_time = cv2.getTickCount() / cv2.getTickFrequency()
    
    # Detect faces
    boxes, probs = mtcnn.detect(frame)
    if boxes is not None:
        for box, prob in zip(boxes, probs):
            if prob < 0.9:
                continue
            
            x1, y1, x2, y2 = map(int, box)
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(frame.shape[1], x2), min(frame.shape[0], y2)
            
            if x1 < x2 and y1 < y2:
                face_crop = frame[y1:y2, x1:x2]
                
                if face_crop.size > 0:
                    try:
                        face_tensor = preprocess(face_crop).unsqueeze(0).to(device)
                        current_face_tensor = face_tensor
                        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        
                        if not save_mode and not show_duplicate_alert:
                            cv2.putText(display_frame, 'Press "s" to save this face', 
                                        (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
                    except Exception as e:
                        print(f"Error processing face: {e}")
    
    if save_mode:
        overlay = display_frame.copy()
        cv2.rectangle(overlay, (40, 35), (600, 120), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.5, display_frame, 0.5, 0, display_frame)
        cv2.putText(display_frame, f'Enter name: {input_name}_', 
                    (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(display_frame, 'Press ENTER to confirm or ESC to cancel', 
                    (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    
    if show_duplicate_alert and current_time - alert_start_time < alert_duration:
        overlay = display_frame.copy()
        cv2.rectangle(overlay, (40, 35), (600, 120), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, display_frame, 0.3, 0, display_frame)
        cv2.putText(display_frame, f"User already registered as '{duplicate_name}'", 
                    (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('Face Recognition - Save Face', display_frame)
    key = cv2.waitKey(1) & 0xFF
    
    if save_mode:
        if key == 13 and input_name and current_face_tensor is not None:
            success, existing_name = save_face_embedding(current_face_tensor, input_name, saved_embeddings)
            if success:
                saved_embeddings = load_embeddings()
            else:
                show_duplicate_alert, duplicate_name, alert_start_time = True, existing_name, current_time
            save_mode, input_name = False, ""
        elif key == 27:
            save_mode, input_name = False, ""
        elif key == 8:
            input_name = input_name[:-1]
        elif 32 <= key <= 126:
            input_name += chr(key)
    elif key == ord('s') and current_face_tensor is not None:
        save_mode = True
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# import torch
# import cv2
# import numpy as np
# from facenet_pytorch import MTCNN, InceptionResnetV1
# from scipy.spatial.distance import cosine
# from torchvision import transforms

# # Initialize MTCNN (Face Detection)
# mtcnn = MTCNN()

# # Initialize Inception Resnet (Face Recognition)
# model = InceptionResnetV1(pretrained='vggface2').eval()

# # Load the saved face embedding (if it exists)
# saved_embedding = torch.load('my_face_embeddings.pth')

# # Load the webcam
# cap = cv2.VideoCapture(0)

# # Threshold for face recognition (cosine distance threshold)
# threshold = 0.6

# # Image preprocessing pipeline
# preprocess = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((160, 160)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255])  # Normalize to [-1, 1]
# ])

# # Function to recognize face by comparing embeddings
# def recognize_face(face_tensor):
#     # Get the face embedding
#     embedding = model(face_tensor)
    
#     # Flatten both the saved embedding and the current face embedding to 1D
#     embedding_flat = embedding.flatten()
#     saved_embedding_flat = saved_embedding.flatten()
    
#     # Calculate the cosine distance between saved embedding and the current face embedding
#     distance = cosine(saved_embedding_flat.detach().numpy(), embedding_flat.detach().numpy())
#     print(f'Cosine distance: {distance}')
    
#     if distance < threshold:
#         print("User recognized: Gaindu")  # Print the recognized user on the console
#         return "Gaindu"
#     else:
#         print("User recognized: Unknown")  # Print "Unknown" on the console if not recognized
#         return "Unknown"

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Detect faces
#     faces, _ = mtcnn.detect(frame)
    
#     if faces is not None:
#         for face in faces:
#             # Draw bounding box around the detected face
#             cv2.rectangle(frame, 
#                           (int(face[0]), int(face[1])), 
#                           (int(face[2]), int(face[3])), 
#                           (0, 255, 0), 2)
            
#             # Crop the face from the frame
#             face_crop = frame[int(face[1]):int(face[3]), int(face[0]):int(face[2])]
            
#             if face_crop.size != 0:
#                 # Preprocess the cropped face
#                 face_tensor = preprocess(face_crop)

#                 # Add batch dimension (for model compatibility)
#                 face_tensor = face_tensor.unsqueeze(0)

#                 # Recognize the face
#                 label = recognize_face(face_tensor)

#                 # Display the label on the image
#                 cv2.putText(frame, label, (int(face[0]), int(face[1]) - 10), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

#         # Show the frame with bounding boxes and labels
#         cv2.imshow('Face Recognition', frame)

#     # Exit when the 'Esc' key is pressed (key code 27)
#     if cv2.waitKey(1) & 0xFF == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()


In [ ]:
import torch
import cv2
import numpy as np
import os
import winsound  # Windows beep sound (use another method for Linux/Mac)
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine
from torchvision import transforms

# Initialize MTCNN with higher confidence threshold to reduce false positives
mtcnn = MTCNN(thresholds=[0.7, 0.8, 0.8])

# Initialize Inception Resnet (Face Recognition)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Directory where face embeddings are stored
embeddings_dir = 'face_embeddings'

# Load the webcam
cap = cv2.VideoCapture(0)

# Threshold for face recognition (cosine distance threshold)
threshold = 0.6

# Image preprocessing pipeline
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Function to load all saved face embeddings
def load_embeddings():
    embeddings = {}
    if os.path.exists(embeddings_dir):
        for filename in os.listdir(embeddings_dir):
            if filename.endswith('.pth'):
                name = os.path.splitext(filename)[0]
                path = os.path.join(embeddings_dir, filename)
                embeddings[name] = torch.load(path, map_location=device)
    return embeddings

# Load saved embeddings
saved_embeddings = load_embeddings()
print(f"Loaded {len(saved_embeddings)} face embeddings")

# Function to recognize face
def recognize_face(face_tensor):
    with torch.no_grad():
        embedding = model(face_tensor.to(device)).detach().cpu()
    
    min_distance = float('inf')
    recognized_name = "Unknown"
    
    for name, saved_embedding in saved_embeddings.items():
        saved_embedding = saved_embedding.cpu() if isinstance(saved_embedding, torch.Tensor) else torch.tensor(saved_embedding)
        distance = cosine(embedding.flatten().numpy(), saved_embedding.flatten().numpy())
        
        if distance < min_distance:
            min_distance = distance
            recognized_name = name
    
    return (recognized_name, min_distance) if min_distance < threshold else ("Unknown", min_distance)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    display_frame = frame.copy()
    boxes, probs = mtcnn.detect(frame, landmarks=False)
    
    if boxes is not None and len(boxes) > 0:
        valid_indices = [i for i, prob in enumerate(probs) if prob is not None and prob > 0.9]
        valid_boxes = [boxes[i] for i in valid_indices]
        
        for box in valid_boxes:
            x1, y1, x2, y2 = [int(coord) for coord in box]
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
            
            if x1 < x2 and y1 < y2:
                face_crop = frame[y1:y2, x1:x2]
                
                if face_crop.size > 0:
                    try:
                        face_tensor = preprocess(face_crop).unsqueeze(0)
                        label, distance = recognize_face(face_tensor)
                        
                        box_color = (0, 255, 0) if label != "Unknown" else (0, 0, 255)
                        cv2.rectangle(display_frame, (x1, y1), (x2, y2), box_color, 2)
                        
                        label_text = f"{label} ({distance:.2f})" if label != "Unknown" else "Unknown"
                        text_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
                        cv2.rectangle(display_frame, 
                                     (x1, y1 - text_size[1] - 10), 
                                     (x1 + text_size[0], y1), 
                                     box_color, -1)
                        cv2.putText(display_frame, label_text, (x1, y1 - 5), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                        
                        if label == "Unknown":
                            print("ALERT: Unrecognized user detected!")
                            winsound.Beep(1000, 500)  # Windows beep (1000 Hz, 500 ms)
                            
                    except Exception as e:
                        print(f"Error processing face: {e}")

    cv2.imshow('Face Recognition', display_frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()